In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [15]:
players_df = pd.read_csv("players_21.csv")
num_rows = players_df.shape[0]
num_col = players_df.shape[1]

print("rows: {}, cols: {}".format(num_rows,num_col))

rows: 18944, cols: 106


In [16]:
pd.set_option('display.max_columns',None)
players_df.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club_name,league_name,league_rank,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,33,1987-06-24,170,72,Argentina,FC Barcelona,Spain Primera Division,1.0,93,93,67500000,560000,"RW, ST, CF",Left,5,4,4,Medium/Low,Messi,Yes,138400000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...",CAM,10.0,NaN,2004-07-01,2021.0,RW,10.0,85.0,92.0,91.0,95.0,38.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,"Finesse Shot, Long Shot Taker (AI), Speed Drib...",85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,NaN,35,24,6,11,15,14,8,89+3,89+3,89+3,92+0,93+0,93+0,93+0,92+0,93+0,93+0,93+0,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,35,1985-02-05,187,83,Portugal,Juventus,Italian Serie A,1.0,92,92,46000000,220000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,75900000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...",LS,7.0,NaN,2018-07-10,2022.0,LS,7.0,89.0,93.0,81.0,89.0,35.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,"Power Free-Kick, Flair, Long Shot Taker (AI), ...",84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,NaN,32,24,7,11,15,14,11,91+1,91+1,91+1,89+0,91+0,91+0,91+0,89+0,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
2,200389,https://sofifa.com/player/200389/jan-oblak/210002,J. Oblak,Jan Oblak,27,1993-01-07,188,87,Slovenia,Atlético Madrid,Spain Primera Division,1.0,91,93,75000000,125000,GK,Right,3,3,1,Medium/Medium,PLAYER_BODY_TYPE_259,Yes,159400000.0,NaN,GK,13.0,NaN,2014-07-16,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,92.0,78.0,90.0,52.0,90.0,"GK Long Throw, Comes For Crosses",13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,NaN,12,18,87,92,78,90,90,33+3,33+3,33+3,32+0,35+0,35+0,35+0,32+0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3
3,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,31,1988-08-21,184,80,Poland,FC Bayern München,German 1. Bundesliga,1.0,91,91,80000000,240000,ST,Right,4,4,4,High/Medium,PLAYER_BODY_TYPE_276,Yes,132000000.0,"#Distance Shooter, #Clinical Finisher",ST,9.0,NaN,2014-07-01,2023.0,NaN,NaN,78.0,91.0,78.0,85.0,43.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,"Solid Player, Finesse Shot, Outside Foot Shot,...",71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,NaN,42,19,15,6,12,8,10,89+2,89+2,89+2,85+0,87+0,87+0,87+0,85+0,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos 

In [17]:
columns_to_be_removed = ["sofifa_id", "player_url", "long_name", "dob", "league_rank", "body_type", "real_face", "release_clause_eur", 
                         "loaned_from", "joined", "contract_valid_until", "team_jersey_number", "nation_jersey_number", "gk_diving", "gk_handling", 
                         "gk_kicking", "gk_reflexes", "gk_speed", "gk_positioning", "goalkeeping_diving", "goalkeeping_handling", "goalkeeping_kicking",
                           "goalkeeping_positioning", "goalkeeping_reflexes"]

players_df.drop(columns=columns_to_be_removed, axis=1, inplace=True)

players_df.head()

,short_name,age,height_cm,weight_kg,nationality,club_name,league_name,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,player_tags,team_position,nation_position,pace,shooting,passing,dribbling,defending,physic,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,L. Messi,33,170,72,Argentina,FC Barcelona,Spain Primera Division,93,93,67500000,560000,"RW, ST, CF",Left,5,4,4,Medium/Low,"#Dribbler, #Distance Shooter, #FK Specialist, ...",CAM,RW,85.0,92.0,91.0,95.0,38.0,65.0,"Finesse Shot, Long Shot Taker (AI), Speed Drib...",85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,NaN,35,24,89+3,89+3,89+3,92+0,93+0,93+0,93+0,92+0,93+0,93+0,93+0,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,Cristiano Ronaldo,35,187,83,Portugal,Juventus,Italian Serie A,92,92,46000000,220000,"ST, LW",Right,5,4,5,High/Low,"#Aerial Threat, #Dribbler, #Distance Shooter, ...",LS,LS,89.0,93.0,81.0,89.0,35.0,77.0,"Power Free-Kick, Flair, Long Shot Taker (AI), ...",84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,NaN,32,24,91+1,91+1,91+1,89+0,91+0,91+0,91+0,89+0,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
2,J. Oblak,27,188,87,Slovenia,Atlético Madrid,Spain Primera Division,91,93,75000000,125000,GK,Right,3,3,1,Medium/Medium,NaN,GK,GK,NaN,NaN,NaN,NaN,NaN,NaN,"GK Long Throw, Comes For Crosses",13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,NaN,12,18,33+3,33+3,33+3,32+0,35+0,35+0,35+0,32+0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3
3,R. Lewandowski,31,184,80,Poland,FC Bayern München,German 1. Bundesliga,91,91,80000000,240000,ST,Right,4,4,4,High/Medium,"#Distance Shooter, #Clinical Finisher",ST,NaN,78.0,91.0,78.0,85.0,43.0,82.0,"Solid Player, Finesse Shot, Outside Foot Shot,...",71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,NaN,42,19,89+2,89+2,89+2,85+0,87+0,87+0,87+0,85+0,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,Neymar Jr,28,175,68,Brazil,Paris Saint-Germain,French Ligue 1,91,91,90000000,270000,"LW, CAM",Right,5,5,5,High/Medium,"#Speedster, #Dribbler, #Playmaker, #FK Special...",LW,NaN,91.0,85.0,86.0,94.0,36.0,59.0,"Injury Prone, Flair, Speed Dribbler (AI), Outs...",85,87,62,87,87,95,88,89,81,95,94,89,96,91,83,80,62,81,50,84,51,36,87,90,92,93,NaN,30,29,84+3,84+3,84+3,90+0,89+0,89+0,89+0,90+0,90+1,90+1,90+1,90+1,83+3,83+3,83+3,90+1,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3


In [18]:
positions_split = players_df['player_positions'].str.split(',').apply(lambda x: [pos.strip() for pos in x])
positions_exploded = positions_split.explode()

encoder = OneHotEncoder(sparse_output=False)

encoded_positions = encoder.fit_transform(positions_exploded.to_numpy().reshape(-1, 1))

position_columns = encoder.get_feature_names_out(['positions'])
encoded_positions_df = pd.DataFrame(encoded_positions, columns=position_columns, index=positions_exploded.index)
encoded_positions_df = encoded_positions_df.groupby(encoded_positions_df.index).max()

position_col_index = players_df.columns.get_loc('player_positions')
df_before = players_df.iloc[:,:position_col_index+1]
df_after = players_df.iloc[:,position_col_index+1:]

players_df = pd.concat([df_before, encoded_positions_df, df_after], axis=1)

players_df.drop(columns=['player_positions'], axis=1, inplace=True)

players_df.head()

,short_name,age,height_cm,weight_kg,nationality,club_name,league_name,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,player_tags,team_position,nation_position,pace,shooting,passing,dribbling,defending,physic,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,L. Messi,33,170,72,Argentina,FC Barcelona,Spain Primera Division,93,93,67500000,560000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5,4,4,Medium/Low,"#Dribbler, #Distance Shooter, #FK Specialist, ...",CAM,RW,85.0,92.0,91.0,95.0,38.0,65.0,"Finesse Shot, Long Shot Taker (AI), Speed Drib...",85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,NaN,35,24,89+3,89+3,89+3,92+0,93+0,93+0,93+0,92+0,93+0,93+0,93+0,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,Cristiano Ronaldo,35,187,83,Portugal,Juventus,Italian Serie A,92,92,46000000,220000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5,4,5,High/Low,"#Aerial Threat, #Dribbler, #Distance Shooter, ...",LS,LS,89.0,93.0,81.0,89.0,35.0,77.0,"Power Free-Kick, Flair, Long Shot Taker (AI), ...",84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,NaN,32,24,91+1,91+1,91+1,89+0,91+0,91+0,91+0,89+0,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
2,J. Oblak,27,188,87,Slovenia,Atlético Madrid,Spain Primera Division,91,93,75000000,125000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,3,3,1,Medium/Medium,NaN,GK,GK,NaN,NaN,NaN,NaN,NaN,NaN,"GK Long Throw, Comes For Crosses",13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,NaN,12,18,33+3,33+3,33+3,32+0,35+0,35+0,35+0,32+0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3
3,R. Lewandowski,31,184,80,Poland,FC Bayern München,German 1. Bundesliga,91,91,80000000,240000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4,4,4,High/Medium,"#Distance Shooter, #Clinical Finisher",ST,NaN,78.0,91.0,78.0,85.0,43.0,82.0,"Solid Player, Finesse Shot, Outside Foot Shot,...",71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,NaN,42,19,89+2,89+2,89+2,85+0,87+0,87+0,87+0,85+0,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,Neymar Jr,28,175,68,Brazil,Paris Saint-Germain,French Ligue 1,91,91,90000000,270000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,5,5,5,High/Medium,"#Speedster, #Dribbler, #Playmaker, #FK Special...",LW,NaN,91.0,85.0,86.0,94.0,36.0,59.0,"Injury Prone, Flair, Speed Dribbler (AI), Outs...",85,87,62,87,87,95,88,89,81,95,94,89,96,91,83,80,62,81,50,84,51,36,87,90,92,93,NaN,30,29,84+3,84+3,84+3,90+0,89+0,89+0,89+0,90+0,90+1,90+1,90+1,90+1,83+3,83+3,83+3,90+1,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3


In [19]:
players_df.drop(columns=['team_position', 'nation_position'], axis=1, inplace=True)
null_val = players_df.isnull().sum()
null_val = null_val[null_val > 0]
null_val

club_name              225
league_name            225
player_tags          17536
pace                  2083
shooting              2083
passing               2083
dribbling             2083
defending             2083
physic                2083
player_traits        10629
defending_marking    18944
dtype: int64

In [20]:
players_df = players_df.dropna(subset=['club_name','league_name','pace','shooting','passing','dribbling','defending','physic'])
null_val = players_df.isnull().sum()
null_val = null_val[null_val > 0]
null_val

player_tags          15281
player_traits         9199
defending_marking    16666
dtype: int64

In [21]:
players_df.drop(columns=['defending_marking'], axis=1, inplace=True)
null_val = players_df.isnull().sum()
null_val = null_val[null_val > 0]
null_val

player_tags      15281
player_traits     9199
dtype: int64

In [22]:
players_df['player_tags'] = players_df['player_tags'].fillna("#no_tag")
players_df['player_traits'] = players_df['player_traits'].fillna("no_trait")
null_val = players_df.isnull().sum()
null_val = null_val[null_val > 0]
null_val

Series([], dtype: int64)

In [23]:
players_tags_split = players_df['player_tags'].str.split(',').apply(lambda x: [pos.strip() for pos in x])
players_tags_exploded = players_tags_split.explode()

encoder = OneHotEncoder(sparse_output=False)

encoded_tags = encoder.fit_transform(players_tags_exploded.to_numpy().reshape(-1,1))

tags_columns = encoder.get_feature_names_out([''])
players_tags_df = pd.DataFrame(encoded_tags, columns=tags_columns, index=players_tags_exploded.index)
players_tags_df = players_tags_df.groupby(players_tags_df.index).max()

tags_col_pos = players_df.columns.get_loc('player_tags')
df_before = players_df.iloc[:, :tags_col_pos]
df_after = players_df.iloc[:, tags_col_pos+1:]

players_df = pd.concat([df_before, players_tags_df, df_after], axis=1)
players_df.head()

,short_name,age,height_cm,weight_kg,nationality,club_name,league_name,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,L. Messi,33,170,72,Argentina,FC Barcelona,Spain Primera Division,93,93,67500000,560000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5,4,4,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,"Finesse Shot, Long Shot Taker (AI), Speed Drib...",85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,35,24,89+3,89+3,89+3,92+0,93+0,93+0,93+0,92+0,93+0,93+0,93+0,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,Cristiano Ronaldo,35,187,83,Portugal,Juventus,Italian Serie A,92,92,46000000,220000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5,4,5,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,"Power Free-Kick, Flair, Long Shot Taker (AI), ...",84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,32,24,91+1,91+1,91+1,89+0,91+0,91+0,91+0,89+0,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
3,R. Lewandowski,31,184,80,Poland,FC Bayern München,German 1. Bundesliga,91,91,80000000,240000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4,4,4,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,"Solid Player, Finesse Shot, Outside Foot Shot,...",71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,42,19,89+2,89+2,89+2,85+0,87+0,87+0,87+0,85+0,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,Neymar Jr,28,175,68,Brazil,Paris Saint-Germain,French Ligue 1,91,91,90000000,270000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,5,5,5,High/Medium,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,"Injury Prone, Flair, Speed Dribbler (AI), Outs...",85,87,62,87,87,95,88,89,81,95,94,89,96,91,83,80,62,81,50,84,51,36,87,90,92,93,30,29,84+3,84+3,84+3,90+0,89+0,89+0,89+0,90+0,90+1,90+1,90+1,90+1,83+3,83+3,83+3,90+1,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3
5,K. De Bruyne,29,181,70,Belgium,Manchester City,English Premier League,91,91,87000000,370000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,4,5,4,High/High,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,86.0,93.0,88.0,64.0,78.0,"Injury Prone, Leadership, Early Crosser, Long ...",94,82,55,94,82,88,85,83,93,92,77,76,78,91,76,91,63,89,74,91,76,66,88,94,84,91,65,53,83+3,83+3,83+3,88+0,88+0,88+0,88+0,88+0,89+2,89+2,89+2,89+2,89+2,89+2,89+2,89+2,79+3,80+3,80+3,80+3,79+3,7

In [24]:
players_traits_split = players_df['player_traits'].str.split(',').apply(lambda x: [pos.strip() for pos in x])
players_traits_exploded = players_traits_split.explode()

encoder = OneHotEncoder(sparse_output=False)

encoded_traits = encoder.fit_transform(players_traits_exploded.to_numpy().reshape(-1,1))

traits_columns = encoder.get_feature_names_out([''])
players_traits_df = pd.DataFrame(encoded_traits, columns=traits_columns, index=players_traits_exploded.index)
players_traits_df = players_traits_df.groupby(players_traits_df.index).max()

traits_col_pos = players_df.columns.get_loc('player_traits')
df_before = players_df.iloc[:, :traits_col_pos]
df_after = players_df.iloc[:, traits_col_pos+1:]

players_df = pd.concat([df_before, players_traits_df, df_after], axis=1)
players_df.head()

,short_name,age,height_cm,weight_kg,nationality,club_name,league_name,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,L. Messi,33,170,72,Argentina,FC Barcelona,Spain Primera Division,93,93,67500000,560000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5,4,4,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,35,24,89+3,89+3,89+3,92+0,93+0,93+0,93+0,92+0,93+0,93+0,93+0,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,Cristiano Ronaldo,35,187,83,Portugal,Juventus,Italian Serie A,92,92,46000000,220000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5,4,5,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,32,24,91+1,91+1,91+1,89+0,91+0,91+0,91+0,89+0,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
3,R. Lewandowski,31,184,80,Poland,FC Bayern München,German 1. Bundesliga,91,91,80000000,240000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4,4,4,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,42,19,89+2,89+2,89+2,85+0,87+0,87+0,87+0,85+0,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,Neymar Jr,28,175,68,Brazil,Paris Saint-Germain,French Ligue 1,91,91,90000000,270000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,5,5,5,High/Medium,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,85,87,62,87,87,95,88,89,81,95,94,89,96,91,83,80,62,81,50,84,51,36,87,90,92,93,30,29,84+3,84+3,84+3,90+0,89+0,89+0,89+0,90+0,90+1,90+1,90+1,90+1,83+3,83+3,83+3,90+1,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3
5,K. De Bruyne,29,181,70,Belgium,Manchester City,English Premier L

In [28]:
columns_to_be_removed = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm',
                         'rwb','lb','lcb','cb','rcb','rb']

players_df.drop(columns=columns_to_be_removed, axis=1, inplace=True)
players_df.head()

,short_name,age,height_cm,weight_kg,nationality,club_name,league_name,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33,170,72,Argentina,FC Barcelona,Spain Primera Division,93,93,67500000,560000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5,4,4,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,35,24
1,Cristiano Ronaldo,35,187,83,Portugal,Juventus,Italian Serie A,92,92,46000000,220000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5,4,5,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,32,24
3,R. Lewandowski,31,184,80,Poland,FC Bayern München,German 1. Bundesliga,91,91,80000000,240000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4,4,4,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,42,19
4,Neymar Jr,28,175,68,Brazil,Paris Saint-Germain,French Ligue 1,91,91,90000000,270000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,5,5,5,High/Medium,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,85,87,62,87,87,95,88,89,81,95,94,89,96,91,83,80,62,81,50,84,51,36,87,90,92,93,30,29
5,K. De Bruyne,29,181,70,Belgium,Manchester City,English Premier League,91,91,87000000,370000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,4,5,4,High/High,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,86.0,93.0,88.0,64.0,78.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94,82,55,94,82,88,85,83,93,92,77,76,78,91,76,91,63,89,74,91,76,66,88,94,84,91,65,53


In [40]:
def emb_split(col_name):
    label_encoder = LabelEncoder()
    players_df[col_name] = label_encoder.fit_transform(players_df[col_name])
    X_train, X_val, y_train, y_val = train_test_split(players_df[col_name], players_df['overall'], test_size=0.2)
    return X_train, X_val, y_train, y_val

In [65]:
def train_emb_col(emb_dim, col_name):
    num_categ = players_df[col_name].nunique()
    X_train, X_val, y_train, y_val = emb_split(col_name)
    best_loss = float('inf')
    best_dim = None
    best_model = None
    for i in emb_dim:
        input_categ = tf.keras.Input(shape=(1,), dtype=tf.int32)
        embedding = tf.keras.layers.Embedding(input_dim=num_categ, output_dim=i, name='embedding_layer')(input_categ)
        output = tf.keras.layers.Flatten()(embedding)
        model = tf.keras.Model(inputs=input_categ, outputs=output)
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=10)
        val_loss = model.evaluate(X_val, y_val)
        if val_loss < best_loss:
            best_loss = val_loss
            best_dim = i
            best_model = model
    return best_model, best_dim

In [66]:
def emb_df(col_name):
    model, best_dim = train_emb_col([2,4,8,16],col_name)
    emb_layer = model.get_layer('embedding_layer')
    emb_wt = emb_layer.get_weights()[0]
    emb_vec = players_df[col_name].apply(lambda x: emb_wt[x])
    emb_df = pd.DataFrame(emb_vec.tolist(), columns=[f'{col_name}_emb_{i}' for i in range(best_dim)])
    return emb_df

In [ ]:
club_emb_df = emb_df('club_name')

In [ ]:
nation_emb_df = emb_df('nationality')

In [ ]:
league_emb_df = emb_df('league_name')

In [82]:
club_col_pos = players_df.columns.get_loc('club_name')
df_before = players_df.iloc[:, :club_col_pos]
df_after = players_df.iloc[:, club_col_pos+1:]
players_df = pd.concat([df_before,club_emb_df,df_after], axis=1)
players_df.head()

,short_name,age,height_cm,weight_kg,nationality,club_name_emb_0,club_name_emb_1,league_name,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33.0,170.0,72.0,6.0,0.955850,0.950151,42.0,93.0,93.0,67500000.0,560000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5.0,4.0,4.0,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85.0,95.0,70.0,91.0,88.0,96.0,93.0,94.0,91.0,96.0,91.0,80.0,91.0,94.0,95.0,86.0,68.0,72.0,69.0,94.0,44.0,40.0,93.0,95.0,75.0,96.0,35.0,24.0
1,Cristiano Ronaldo,35.0,187.0,83.0,122.0,0.954205,0.936377,25.0,92.0,92.0,46000000.0,220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5.0,4.0,5.0,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84.0,95.0,90.0,82.0,86.0,88.0,81.0,76.0,77.0,92.0,87.0,91.0,87.0,95.0,71.0,94.0,95.0,84.0,78.0,93.0,63.0,29.0,95.0,82.0,84.0,95.0,32.0,24.0
3,R. Lewandowski,31.0,184.0,80.0,121.0,0.945136,0.920557,20.0,91.0,91.0,80000000.0,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4.0,4.0,4.0,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71.0,94.0,85.0,84.0,89.0,85.0,79.0,85.0,70.0,88.0,77.0,78.0,77.0,93.0,82.0,89.0,84.0,76.0,86.0,85.0,81.0,49.0,94.0,79.0,88.0,88.0,42.0,19.0
4,Neymar Jr,28.0,175.0,68.0,20.0,1.033333,1.013870,18.0,91.0,91.0,90000000.0,270000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,5.0,5.0,5.0,High/Medium,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,91.0,85.0,86.0,94.0,36.0,59.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,85.0,87.0,62.0,87.0,87.0,95.0,88.0,89.0,81.0,95.0,94.0,89.0,96.0,91.0,83.0,80.0,62.0,81.0,50.0,84.0,51.0,36.0,87.0,90.0,92.0,93.0,30.0,29.0
5,K. De Bruyne,29.0,181.0,70.0,14.0,0.945136,0.920557,16.0,91.0,91.0,87000000.0,370000.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,4.0,5.0,4.0,High/High,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,86.0,93.0,88.0,64.0,78.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93

In [83]:
nation_col_pos = players_df.columns.get_loc('nationality')
df_before = players_df.iloc[:, :nation_col_pos]
df_after = players_df.iloc[:, nation_col_pos+1:]
players_df = pd.concat([df_before,nation_emb_df,df_after], axis=1)
players_df.head()

,short_name,age,height_cm,weight_kg,nationality_emb_0,nationality_emb_1,nationality_emb_2,nationality_emb_3,nationality_emb_4,nationality_emb_5,nationality_emb_6,nationality_emb_7,nationality_emb_8,nationality_emb_9,nationality_emb_10,nationality_emb_11,nationality_emb_12,nationality_emb_13,nationality_emb_14,nationality_emb_15,club_name_emb_0,club_name_emb_1,league_name,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33.0,170.0,72.0,5.267376,5.253035,5.307324,5.277308,5.228816,5.278159,5.234392,5.307262,5.229340,5.316258,5.310812,5.278153,5.241920,5.289532,5.261187,5.272172,0.955850,0.950151,42.0,93.0,93.0,67500000.0,560000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5.0,4.0,4.0,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85.0,95.0,70.0,91.0,88.0,96.0,93.0,94.0,91.0,96.0,91.0,80.0,91.0,94.0,95.0,86.0,68.0,72.0,69.0,94.0,44.0,40.0,93.0,95.0,75.0,96.0,35.0,24.0
1,Cristiano Ronaldo,35.0,187.0,83.0,3.211513,3.206018,3.281048,3.282329,3.264278,3.253104,3.240057,3.215362,3.259876,3.265996,3.263393,3.216079,3.233986,3.238109,3.297438,3.209075,0.954205,0.936377,25.0,92.0,92.0,46000000.0,220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5.0,4.0,5.0,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84.0,95.0,90.0,82.0,86.0,88.0,81.0,76.0,77.0,92.0,87.0,91.0,87.0,95.0,71.0,94.0,95.0,84.0,78.0,93.0,63.0,29.0,95.0,82.0,84.0,95.0,32.0,24.0
3,R. Lewandowski,31.0,184.0,80.0,5.009506,5.100152,5.096868,5.034960,5.019358,5.107184,5.046790,5.075832,5.074919,5.106402,5.100490,5.065495,5.092236,5.077969,5.091897,5.105593,0.945136,0.920557,20.0,91.0,91.0,80000000.0,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4.0,4.0,4.0,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71.0,94.0,85.0,84.0,89.0,85.0,79.0,85.0,70.0,88.0,77.0,78.0,77.0,93.0,82.0,89.0,84.0,76.0,86.0,85.0,81.0,49.0,94.0,79.0,88.0,88.0,42.0,19.0
4,Neymar Jr,28.0,175.0,68.0,2.904723,2.920398,2.919375,2.971099,2.918270,2.910767,2.967917,2.907134,2.924917,2.954134,2.913753,2.981256,2.950684,2.974055,2.958389,2.911059,1.033333,1.013870,18.0,91.0,91.0,9000

In [84]:
league_col_pos = players_df.columns.get_loc('league_name')
df_before = players_df.iloc[:, :league_col_pos]
df_after = players_df.iloc[:, league_col_pos+1:]
players_df = pd.concat([df_before,league_emb_df,df_after], axis=1)
players_df.head()

,short_name,age,height_cm,weight_kg,nationality_emb_0,nationality_emb_1,nationality_emb_2,nationality_emb_3,nationality_emb_4,nationality_emb_5,nationality_emb_6,nationality_emb_7,nationality_emb_8,nationality_emb_9,nationality_emb_10,nationality_emb_11,nationality_emb_12,nationality_emb_13,nationality_emb_14,nationality_emb_15,club_name_emb_0,club_name_emb_1,league_name_emb_0,league_name_emb_1,league_name_emb_2,league_name_emb_3,overall,potential,value_eur,wage_eur,positions_CAM,positions_CB,positions_CDM,positions_CF,positions_CM,positions_GK,positions_LB,positions_LM,positions_LW,positions_LWB,positions_RB,positions_RM,positions_RW,positions_RWB,positions_ST,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,_#Acrobat,_#Aerial Threat,_#Clinical Finisher,_#Complete Defender,_#Complete Forward,_#Complete Midfielder,_#Crosser,_#Distance Shooter,_#Dribbler,_#Engine,_#FK Specialist,_#Playmaker,_#Poacher,_#Speedster,_#Strength,_#Tackling,_#Tactician,_#no_tag,pace,shooting,passing,dribbling,defending,physic,_Chip Shot (AI),_Dives Into Tackles (AI),_Early Crosser,_Finesse Shot,_Flair,_Giant Throw-in,_Injury Prone,_Leadership,_Long Passer (AI),_Long Shot Taker (AI),_Long Throw-in,_One Club Player,_Outside Foot Shot,_Playmaker (AI),_Power Free-Kick,_Power Header,_Solid Player,_Speed Dribbler (AI),_Team Player,_Technical Dribbler (AI),_no_trait,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_standing_tackle,defending_sliding_tackle
0,L. Messi,33.0,170.0,72.0,5.267376,5.253035,5.307324,5.277308,5.228816,5.278159,5.234392,5.307262,5.229340,5.316258,5.310812,5.278153,5.241920,5.289532,5.261187,5.272172,0.955850,0.950151,4.299503,4.346333,4.380443,4.373397,93.0,93.0,67500000.0,560000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Left,5.0,4.0,4.0,Medium/Low,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,92.0,91.0,95.0,38.0,65.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,85.0,95.0,70.0,91.0,88.0,96.0,93.0,94.0,91.0,96.0,91.0,80.0,91.0,94.0,95.0,86.0,68.0,72.0,69.0,94.0,44.0,40.0,93.0,95.0,75.0,96.0,35.0,24.0
1,Cristiano Ronaldo,35.0,187.0,83.0,3.211513,3.206018,3.281048,3.282329,3.264278,3.253104,3.240057,3.215362,3.259876,3.265996,3.263393,3.216079,3.233986,3.238109,3.297438,3.209075,0.954205,0.936377,4.250766,4.246537,4.253244,4.294671,92.0,92.0,46000000.0,220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,5.0,4.0,5.0,High/Low,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,93.0,81.0,89.0,35.0,77.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,84.0,95.0,90.0,82.0,86.0,88.0,81.0,76.0,77.0,92.0,87.0,91.0,87.0,95.0,71.0,94.0,95.0,84.0,78.0,93.0,63.0,29.0,95.0,82.0,84.0,95.0,32.0,24.0
3,R. Lewandowski,31.0,184.0,80.0,5.009506,5.100152,5.096868,5.034960,5.019358,5.107184,5.046790,5.075832,5.074919,5.106402,5.100490,5.065495,5.092236,5.077969,5.091897,5.105593,0.945136,0.920557,4.127083,4.170119,4.149698,4.106336,91.0,91.0,80000000.0,240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Right,4.0,4.0,4.0,High/Medium,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.0,91.0,78.0,85.0,43.0,82.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,71.0,94.0,85.0,84.0,89.0,85.0,79.0,85.0,70.0,88.0,77.0,78.0,77.0,93.0,82.0,89.0,84.0,76.0,86.0,85.0,81.0,49.0,94.0,79.0,88.0,88.0,42.0,19.0
4,Neymar Jr,28.0,175.0,68.0,2.904723,2.920398,2.919375,2